In [ ]:
from envs.secret.secret_envs_wrapper import SecretEnv0
import numpy as np
from rl_algorithms.planning.dyna_q import DynaQBlackboxAgent

env = SecretEnv0()
agent = DynaQBlackboxAgent(n_actions=env.num_actions(), alpha=0.1, gamma=0.99, epsilon=0.1, n_planning=20)

n_episodes = 3000
reward_history = []

for episode in range(n_episodes):
    env.reset()
    state = env.state_id()
    done = env.is_game_over()
    total_reward = 0
    while not done:
        valid_actions = env.available_actions()
        action = agent.select_action(state, valid_actions)
        env.step(action)
        next_state = env.state_id()
        reward = env.score() if env.is_game_over() else 0.0
        done = env.is_game_over()
        next_valid_actions = env.available_actions() if not done else []
        agent.learn_step(
            state, action, reward, next_state,
            valid_next_actions=next_valid_actions,
            get_valid_actions_fn=lambda s: env.available_actions() if not env.is_game_over() else []
        )
        state = next_state
        total_reward += reward
    reward_history.append(total_reward)
print("SecretEnv0 | Mean reward (last 100 episodes):", np.mean(reward_history[-100:]))